# Feature Selection

We will explore the following topics:
- selecting features for classification models
- selecting features for regression models
- using forward and backward feature selection
- using exhaustive feature selection
- eliminating features recursively in a regression model
- eliminating features recursively in a classification model
- using Boruta for feature selection
- using regularization and other embedded methods
- using principal component analysis

## Selecting features for classification models

_Filter methods_ are techniques for determining the _k-best_ features based on their linear and non-linear relationship with the target. 

They are also someitmes called _univariante_ methods since they evaluate the relationship between the feature and the target independent of the impact of other features.

### Mutual information classification for feature selection with a categorical target

We can use __mutual information__ classification or __analysis of variance (ANOVA)__ tests to select features when we have a categorical target. 

__Mutual information__ is a measure of how much information about a variable is provided by knowing the value of another variable. At the extreme, when features are completely independent, the mutual information score is 0.

In [3]:
import pandas as pd
from feature_engine.encoding import OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from data.load import load_nls97compba

In [5]:
nls97compba = load_nls97compba()
feature_cols = [
    "gender",
    "satverbal",
    "satmath",
    "gpascience",
    "gpaenglish",
    "gpamath",
    "gpaoverall",
    "motherhighgrade",
    "fatherhighgrade",
    "parentincome",
]

X_train, X_test, y_train, y_test = train_test_split(
    nls97compba[feature_cols],
    nls97compba[["completedba"]],
    test_size=0.3,
    random_state=0,
)

In [19]:
ohe = OneHotEncoder(drop_last=True, variables=["gender"])
X_train_enc = ohe.fit_transform(X_train)

scaler = StandardScaler()
standcols = X_train_enc.iloc[:, :-1].columns
X_train_enc = pd.DataFrame(
    scaler.fit_transform(X_train_enc[standcols]),
    columns=standcols,
    index=X_train_enc.index,
).join(X_train_enc[["gender_Female"]])

We are ready to select features for our model of bachelor's degree completion, using mutual information classification.

We call `fit` and use the `get_support` method to get the five best features.

In [20]:
ksel = SelectKBest(score_func=mutual_info_classif, k=5)
ksel.fit(X_train_enc, y_train.values.ravel())

sel_cols = X_train_enc.columns[ksel.get_support()]
sel_cols

Index(['satverbal', 'satmath', 'gpascience', 'gpaenglish', 'gpaoverall'], dtype='object')

We can see the score of each features using the `scores_` attribute.

In [15]:
pd.DataFrame(
    {"score": ksel.scores_, "feature": X_train_enc.columns},
    columns=["feature", "score"],
).sort_values(["score"], ascending=False)

,feature,score
3,gpaenglish,0.103650
5,gpaoverall,0.095984
1,satmath,0.072620
0,satverbal,0.063807
2,gpascience,0.044314
4,gpamath,0.041551
6,motherhighgrade,0.036985
8,parentincome,0.031136
7,fatherhighgrade,0.021799
9,gender_Female,0.019019


^ This is a stochastic process, so we will get different results each time we run it.

To get the same results each time, you can pass a partial function to `score_func`:

In [16]:
from functools import partial

ksel = SelectKBest(score_func=partial(mutual_info_classif, random_state=0), k=5)

We can create a DataFrame with just the important features using the `sel_cols` array we created using the `get_support` (or use the `transform` method of `SelectKBest` instead, but it returns array instead of DataFrame). 

In [23]:
X_train_analysis = X_train_enc[sel_cols]
X_train_analysis.dtypes

satverbal     float64
satmath       float64
gpascience    float64
gpaenglish    float64
gpaoverall    float64
dtype: object

### ANOVA F-value for feature selection with a categorical target

Alternatively, we can use ANOVA instead of mutual information. ANOVA evaluates how different the mean for a feature is for each target class. This is a good metric for univariante feature selection when we can assume a _linear relationship_ between features and the target and our features are _normally distributed_. If those assumptions do not hold, __mutual information__ classification is a better choice.

In [25]:
ksel = SelectKBest(score_func=f_classif, k=5)
ksel.fit(X_train_enc, y_train.values.ravel())

sel_cols = X_train_enc.columns[ksel.get_support()]
sel_cols

Index(['satverbal', 'satmath', 'gpascience', 'gpaenglish', 'gpaoverall'], dtype='object')

In [26]:
pd.DataFrame(
    {"score": ksel.scores_, "feature": X_train_enc.columns},
    columns=["feature", "score"],
).sort_values(["score"], ascending=False)

,feature,score
5,gpaoverall,119.471301
3,gpaenglish,108.006221
2,gpascience,96.823929
1,satmath,84.901228
0,satverbal,77.362692
4,gpamath,60.929884
7,fatherhighgrade,37.480915
6,motherhighgrade,29.377457
8,parentincome,22.265728
9,gender_Female,15.098132


^ Showing the scores gives us some indication of whether the selected value for _k_ makes best sense. 

There is a big decline from the sixth to seventh (61-37), suggesting that we should at least consider a value for _k_ of 6.

### Summary

ANOVA tests, and the mutual information classification we did earlier, do not take into account features that are only important in multivariante analysis.

## Selecting features for regression models

__Regression models__ have a continuous target. Two good options are selection based on F-tests and selection based on mutual information for regression.

### F-tests for feature selection with a continuous target

The F-statistic is a measure of the strength of the linear correlation between a target and a single regressor (`f_regression` scoring function in `scikit-learn`). 

In [49]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from data.load import load_nls97wages

In [46]:
nls97wages = load_nls97wages()
feature_cols = [
    "satverbal",
    "satmath",
    "gpascience",
    "gpaenglish",
    "gpamath",
    "gpaoverall",
    "gender",
    "motherhighgrade",
    "fatherhighgrade",
    "parentincome",
    "completedba",
]

X_train, X_test, y_train, y_test = train_test_split(
    nls97wages[feature_cols],
    nls97wages[["wageincome"]],
    test_size=0.3,
    random_state=0,
)

cat_cols = ["gender"]
num_cols = feature_cols[:]
num_cols.remove("gender")

cat_transformer = make_pipeline(OneHotEncoder(drop="first"))
num_transformer = make_pipeline(StandardScaler())

transformer = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer, cat_cols),
        ("num", num_transformer, num_cols),
    ],
    verbose_feature_names_out=False,  # We don't need to prefix every feature with cat_ and num_.
)

X_train_enc = pd.DataFrame(
    transformer.fit_transform(X_train),
    columns=transformer.get_feature_names_out(),
    index=X_train.index,
)
y_train = pd.DataFrame(
    scaler.fit_transform(y_train), columns=["wageincome"], index=y_train.index
)

In [47]:
ksel = SelectKBest(score_func=f_regression, k=5)
ksel.fit(X_train_enc, y_train.values.ravel())
sel_cols = X_train_enc.columns[ksel.get_support()]
sel_cols

Index(['gender_Male', 'satmath', 'gpascience', 'parentincome', 'completedba'], dtype='object')

In [48]:
pd.DataFrame(
    {"score": ksel.scores_, "feature": X_train_enc.columns},
    columns=["feature", "score"],
).sort_values(["score"], ascending=False)

,feature,score
2,satmath,44.812092
10,completedba,37.506374
0,gender_Male,25.941412
9,parentincome,24.374384
3,gpascience,20.532561
1,satverbal,18.887186
6,gpaoverall,17.292617
5,gpamath,12.780981
4,gpaenglish,10.126248
7,motherhighgrade,9.171498


^ The disadvantage of the __F-statistic__ is that it assumes a linear relationship between each feature and the target. When that assumption does not make sense, we can use mutual information for regression instead.

### Mutual information for feature selection with a continuous target

In [50]:
from functools import partial

ksel = SelectKBest(partial(mutual_info_regression, random_state=0), k=5)
ksel.fit(X_train_enc, y_train.values.ravel())

sel_cols = X_train_enc.columns[ksel.get_support()]
sel_cols

Index(['gender_Male', 'satmath', 'gpascience', 'fatherhighgrade',
       'completedba'],
      dtype='object')

In [51]:
pd.DataFrame(
    {"score": ksel.scores_, "feature": X_train_enc.columns},
    columns=["feature", "score"],
).sort_values(["score"], ascending=False)

,feature,score
2,satmath,0.099232
0,gender_Male,0.064096
10,completedba,0.059069
3,gpascience,0.052042
8,fatherhighgrade,0.037278
9,parentincome,0.014786
5,gpamath,0.009263
1,satverbal,0.000000
4,gpaenglish,0.000000
6,gpaoverall,0.000000


^ `parentincome` was selected with F-tests and `fatherhighgrade` with mutual information. Otherwise, the same features are selected.

A key advantage of __mutual information__ for regression compared with __F-tests__ is that it does not assume a linear relationship between the feature and the target.

### Summary

The feature selection methods we have used so far are known as _filter methods_. They examine the univariante relationship betwen each feature and the target. 


## Using forward and backward feature selection

__Forward__ and __backward__ feature selection, as their names suggest, select features by adding them one by one - or subtracting them for backward selection - and assessing the impact on model performance after each iteration. Since both methods assess that performance based on a given algorithm, they are considered __wrapper__ selection methods.


Wrapper feature selection methods have two advantages over the filter method we have explored so far.

- they evaluate the importance of features as other features are included
- since features are evaluated based on their contributions to the performance of specific algorithm, we get a better sense of which features will ultimately matter

The main disadvantage of wrapper methods is that they can be quite expensive computationally since they retrain the model after each iteration.

### Using forward feature selection

__Forward feature selection__ starts by identifying a subset of features that individually have a significant relationship with a target, not unlike the filter methods.


We can use forward feature selection to develop a model of bachelor's degree completion. Since wrapper method require us to choose an algorithm, and this is a binary target, let's use `scikit-learn`'s __random forest classifier__. 

In [57]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from data.load import load_nls97compba

In [60]:
nls97compba = load_nls97compba()

feature_cols = [
    "gender",
    "satverbal",
    "satmath",
    "gpascience",
    "gpaenglish",
    "gpamath",
    "gpaoverall",
    "motherhighgrade",
    "fatherhighgrade",
    "parentincome",
]

X_train, X_test, y_train, y_test = train_test_split(
    nls97compba[feature_cols],
    nls97compba[["completedba"]],
    test_size=0.3,
    random_state=0,
)
cat_columns = ["gender"]
num_columns = feature_cols[:]
num_columns.remove("gender")

cat_transformer = make_pipeline(OneHotEncoder(drop="first"))
num_transformer = make_pipeline(StandardScaler())

transformer = ColumnTransformer(
    [("cat", cat_transformer, cat_columns), ("num", num_transformer, num_columns)],
    verbose_feature_names_out=False,
)

X_train_enc = pd.DataFrame(
    transformer.fit_transform(X_train),
    columns=transformer.get_feature_names_out(),
    index=X_train.index,
)
X_train_enc

,gender_Male,satverbal,satmath,gpascience,gpaenglish,gpamath,gpaoverall,motherhighgrade,fatherhighgrade,parentincome
personid,,,,,,,,,,
185424,0.0,-1.211808,-0.936656,-1.501666,-1.290166,-0.824340,-1.054224,-0.777285,-0.884695,1.007125
162597,0.0,-1.120269,-0.326391,1.085734,0.845885,1.339766,0.827237,0.742526,0.180973,-0.121043
834993,1.0,0.527432,0.981319,0.247060,0.311872,0.146325,-0.025984,0.742526,0.891418,-0.476950
379604,1.0,0.435893,0.719777,-0.912809,0.116068,0.464576,-0.485410,0.362573,0.891418,-0.491161
882321,0.0,-0.754114,-0.675114,-0.127667,-0.364544,0.225888,0.455320,1.122479,-0.174250,0.821076
...,...,...,...,...,...,...,...,...,...,...
932713,1.0,1.351282,0.719777,0.693163,-0.400145,-0.203751,-0.091616,0.742526,-0.884695,1.007125
301273,1.0,0.069737,-0.064849,-1.430289,-0.275542,-0.140101,-0.769817,-0.777285,-0.884695,-0.398136
739668,0.0,-0.662575,-0.239211,-0.484550,-0.560348,-0.299226,-0.660430,0.362573,-0.174250,-0.556476


In [66]:
rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
sfs = SequentialFeatureSelector(rfc, n_features_to_select=5, direction="forward", cv=5)
sfs.fit(X_train_enc, y_train.values.ravel())

SequentialFeatureSelector(estimator=RandomForestClassifier(n_jobs=-1,
                                                           random_state=0),
                          n_features_to_select=5)

In [67]:
sel_cols = X_train_enc.columns[sfs.get_support()]
sel_cols

Index(['gender_Male', 'satmath', 'gpaenglish', 'gpaoverall',
       'fatherhighgrade'],
      dtype='object')

### Summary

One disadvantage of forward selection si that _once a feature is selected, it is not removed, even though it may decline in importance as additional features are added_. 

## Using backward feature selection

Backward feature selection starts with all features and eliminates teh least important. It then repeats this process with the remaining features.

In [71]:
rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
sfs = SequentialFeatureSelector(rfc, n_features_to_select=5, direction="backward", cv=5)
sfs.fit(X_train_enc, y_train.values.ravel())

SequentialFeatureSelector(direction='backward',
                          estimator=RandomForestClassifier(n_jobs=-1,
                                                           random_state=0),
                          n_features_to_select=5)

In [72]:
sel_cols = X_train_enc.columns[sfs.get_support()]
sel_cols

Index(['gender_Male', 'satverbal', 'gpaoverall', 'fatherhighgrade',
       'parentincome'],
      dtype='object')

^ We get different results for out feature selection. `satmath` and `gpaenglish` are no longer selected, and `gpaoverall` and `parentincome` are.

### Summary

Backward feature selection has the opposite drawback to forward feature selection. _Once a feature has been removed, it is not re-evaluated, even though its importance may change with different feature mixtures_. Let's try __exhaustive feature selection__ instead.

## Using exhaustive feature selection

> Note, we are skipping this because this might not be practical in production. 

### Summary

Wrapper methods, such as forward, backward and exhaustive feature selection, tax system resources because they need to be trained with each iteration, and the more difficult the choosen algorithm is to implement, the more this is an issue.


__Recursive feature elimination (RFE)__ is something of a compromise between the simplicity of filter methods and the better information provided by wrapper methods.

## Eliminating features recursively in a regression model

A popular wrapper method is RFE. This method starts with all features, removes the lowest weighted one (based on a coefficient or feature importance measure), and repeats the process until the best-fitting model has been identified. When a feature is removed, it is given a ranking reflecting the point at which it was removed.

RFE can be used for both regression models and classification models.

In [18]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from data.load import load_nls97wages

In [29]:
nls97wages = load_nls97wages()
feature_cols = [
    "satverbal",
    "satmath",
    "gpascience",
    "gpaenglish",
    "gpamath",
    "gpaoverall",
    "motherhighgrade",
    "fatherhighgrade",
    "parentincome",
    "gender",
    "completedba",
]
X_train, X_test, y_train, y_test = train_test_split(
    nls97wages[feature_cols], nls97wages[["weeklywage"]], test_size=0.3, random_state=0
)

In [30]:
num_cols = list(X_train.columns)
num_cols.remove("gender")
cat_cols = ["gender"]

cat_transformer = make_pipeline(OneHotEncoder(drop="first"))
num_transformer = make_pipeline(StandardScaler())

transformer = ColumnTransformer(
    [
        ("cat_cols", cat_transformer, cat_cols),
        ("num_cols", num_transformer, num_cols),
    ],
    verbose_feature_names_out=False,
)
X_train_enc = pd.DataFrame(
    transformer.fit_transform(X_train),
    columns=transformer.get_feature_names_out(),
    index=X_train.index,
)
X_test_enc = pd.DataFrame(
    transformer.fit_transform(X_test),
    columns=transformer.get_feature_names_out(),
    index=X_test.index,
)
scaler = StandardScaler()
y_train = pd.DataFrame(
    scaler.fit_transform(y_train), columns=["weeklywage"], index=y_train.index
)
y_test = pd.DataFrame(
    scaler.fit_transform(y_test), columns=["weeklywage"], index=y_test.index
)

Since RFE is a wrapper method, we need to choose an algorithm around which the selection will be wrapped. Random forests for regression make sense in this case. We are modelling a continuous target and do not want to assume a linear relationship between the features and the target.

In [33]:
# We set max_depth to 2 to avoid overfitting.
rfr = RandomForestRegressor(max_depth=2)
tree_sel = RFE(estimator=rfr, n_features_to_select=5)
tree_sel.fit(X_train_enc, y_train.values.ravel())
sel_cols = X_train_enc.columns[tree_sel.get_support()]
sel_cols

Index(['satmath', 'gpascience', 'gpaoverall', 'parentincome', 'completedba'], dtype='object')

We can use the `ranking_` attribute to see when each of the eliminated features was removed.

In [37]:
pd.DataFrame(
    {
        "ranking": tree_sel.ranking_,
        "feature": X_train_enc.columns,
    },
    columns=["feature", "ranking"],
).sort_values(["ranking"], ascending=True)

,feature,ranking
2,satmath,1
3,gpascience,1
6,gpaoverall,1
9,parentincome,1
10,completedba,1
0,gender_Male,2
7,motherhighgrade,3
8,fatherhighgrade,4
1,satverbal,5
4,gpaenglish,6


We can use the `score` method to get the __r-squared value__, also known as the coefficient of determination. __R-squared__ is a measure of the percentage of total variation explained by our model. We get a very low score, indicating that our model explains only a little of the variation.

See `rfr.score?` for detailed explanation.

In [40]:
rfr.fit(tree_sel.transform(X_train_enc), y_train.values.ravel())
rfr.score(tree_sel.transform(X_test_enc), y_test)

0.11476590269688303

Let's see whether we get any better results using RFE with a linear regression model. This model returns the same features as the random forest regressor.

In [41]:
lr = LinearRegression()
lr_sel = RFE(estimator=lr, n_features_to_select=5)
lr_sel.fit(X_train_enc, y_train.values.ravel())
sel_cols = X_train_enc.columns[tree_sel.get_support()]
sel_cols

Index(['satmath', 'gpascience', 'gpaoverall', 'parentincome', 'completedba'], dtype='object')

In [42]:
lr.fit(lr_sel.transform(X_train_enc), y_train.values.ravel())
lr.score(lr_sel.transform(X_test_enc), y_test)

0.17393446569288262

^ The linear model is not really much better than the random forest model. 

## Eliminating features recursively in a classification model

In [44]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from data.load import load_nls97compba

In [47]:
nls97compba = load_nls97compba()
feature_cols = [
    "satverbal",
    "satmath",
    "gpascience",
    "gpaenglish",
    "gpamath",
    "gpaoverall",
    "gender",
    "motherhighgrade",
    "fatherhighgrade",
    "parentincome",
]
X_train, X_test, y_train, y_test = train_test_split(
    nls97compba[feature_cols],
    nls97compba[["completedba"]],
    test_size=0.3,
    random_state=0,
)

In [51]:
num_cols = feature_cols[:]
num_cols.remove("gender")
cat_cols = ["gender"]

num_transformer = make_pipeline(StandardScaler())
cat_transformer = make_pipeline(OneHotEncoder(drop="first"))

transformer = ColumnTransformer(
    [("num_cols", num_transformer, num_cols), ("cat_cols", cat_transformer, cat_cols)],
    verbose_feature_names_out=False,
)
X_train_enc = pd.DataFrame(
    transformer.fit_transform(X_train),
    columns=transformer.get_feature_names_out(),
    index=X_train.index,
)
X_test_enc = pd.DataFrame(
    transformer.fit_transform(X_test),
    columns=transformer.get_feature_names_out(),
    index=X_test.index,
)

In [54]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=2, n_jobs=-1, random_state=0)
tree_sel = RFE(estimator=rfc, n_features_to_select=5)
tree_sel.fit(X_train_enc, y_train.values.ravel())
sel_cols = X_train_enc.columns[tree_sel.get_support()]
sel_cols

Index(['satverbal', 'satmath', 'gpascience', 'gpaenglish', 'gpaoverall'], dtype='object')

In [57]:
pd.DataFrame(
    {
        "feature": X_train_enc.columns,
        "ranking": tree_sel.ranking_,
    },
    columns=["feature", "ranking"],
).sort_values(["ranking"], ascending=True)

,feature,ranking
0,satverbal,1
1,satmath,1
2,gpascience,1
3,gpaenglish,1
5,gpaoverall,1
4,gpamath,2
8,parentincome,3
7,fatherhighgrade,4
6,motherhighgrade,5
9,gender_Male,6


In [59]:
rfc.fit(tree_sel.transform(X_train_enc), y_train.values.ravel())
y_pred = rfc.predict(tree_sel.transform(X_test_enc))
accuracy_score(y_test, y_pred)

0.6666666666666666

## Using regularization and other embedded methods

__Regularization__ methods are embedded methods. Like _wrapper methods_, embedded methods evaluate features relative to a given algorithm. But they are not expensive computationally.

Embedded models use the following process:
1. Train a model
2. Estimate each feature's importance to the model's prediction
3. Remove features with low importance

Regularization accomplishes this by adding a penalty to any model to constrain the parameters. __L1 regularization__, also referred to as __lasso regularization__, shrinks some of the coefficients in a regression model to 0, effectively eliminating those features.

### Using L1 Regularization

In [8]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from data.load import load_nls97compba

In [11]:
nls97compba = load_nls97compba()
feature_cols = [
    "satverbal",
    "satmath",
    "gpascience",
    "gpaenglish",
    "gpamath",
    "gpaoverall",
    "gender",
    "motherhighgrade",
    "fatherhighgrade",
    "parentincome",
]
X_train, X_test, y_train, y_test = train_test_split(
    nls97compba[feature_cols],
    nls97compba[["completedba"]],
    test_size=0.3,
    random_state=0,
)


cat_cols = ["gender"]
num_cols = feature_cols[:]
num_cols.remove("gender")

cat_transformer = make_pipeline(OneHotEncoder(drop="first"))
num_transformer = make_pipeline(StandardScaler())

transformer = ColumnTransformer(
    [
        ("num", num_transformer, num_cols),
        ("cat", cat_transformer, cat_cols),
    ],
    verbose_feature_names_out=False,
)
X_train_enc = pd.DataFrame(
    transformer.fit_transform(X_train),
    columns=transformer.get_feature_names_out(),
    index=X_train.index,
)
X_test_enc = pd.DataFrame(
    transformer.fit_transform(X_test),
    columns=transformer.get_feature_names_out(),
    index=X_test.index,
)

In [12]:
lr = LogisticRegression(C=1, penalty="l1", solver="liblinear")
reg_sel = SelectFromModel(lr, max_features=5)
reg_sel.fit(X_train_enc, y_train.values.ravel())
sel_cols = X_train_enc.columns[reg_sel.get_support()]
sel_cols

Index(['satmath', 'gpascience', 'gpaoverall', 'fatherhighgrade',
       'gender_Male'],
      dtype='object')

In [13]:
lr.fit(reg_sel.transform(X_train_enc), y_train.values.ravel())
y_pred = lr.predict(reg_sel.transform(X_test_enc))
accuracy_score(y_test, y_pred)

0.6886446886446886

### Summary

Lasso regularization assumes a linear relationship between the features and the target, which might not be appropriate. Fortunately, there are embedded feature selection methods that do not make that assumption. A good alternative to logistic regression for the embedded model is a random forest classifier.

### Using a random forest classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=2, n_jobs=-1, random_state=0)
rfc_sel = SelectFromModel(rfc, max_features=5)
rfc_sel.fit(X_train_enc, y_train.values.ravel())
sel_cols = X_train_enc.columns[rfc_sel.get_support()]
sel_cols

Index(['satverbal', 'gpascience', 'gpaenglish', 'gpaoverall'], dtype='object')

^ This actually selects very different features from the lasso regression. `satmath`, `fatherhighgrade` and `gender_Male` are no longer selected, while `satverbal` and `gpaenglish` are. This is likely due to the relaxation of the assumption of linearity.

In [19]:
rfc.fit(rfc_sel.transform(X_train_enc), y_train.values.ravel())
y_pred = rfc.predict(rfc_sel.transform(X_test_enc))
accuracy_score(y_test, y_pred)

0.6703296703296703

## Using principal component analysis

PCA allows us to replace the existing feature set with a limited number of components, each of which explains an important amount of the variance. It does this by finding a component that captures the largest amount of variance, followed by a second component that captures the largest amount of remaining variance, and then a third component, and so on. 

One key advantage of this approach is that these components, known as __principal components__, are uncorrelated.

It is better to think of PCA as a tool for dimension reduction rather than feature selection.

In [20]:
from sklearn.decomposition import PCA

In [21]:
pca = PCA(n_components=5)
pca.fit(X_train_enc)

PCA(n_components=5)

The `components_` attribute of PCA object returns the scores of all 10 features on each of the 5 components. The features that drive the first component most are those with scores that have the highest absolute value

In [47]:
components_df = pd.DataFrame(pca.components_, columns=X_train_enc.columns).T
components_df

,0,1,2,3,4
satverbal,-0.344599,-0.155500,-0.609857,-0.015720,-0.194730
satmath,-0.365310,-0.133004,-0.560769,0.101711,0.108085
gpascience,-0.399085,0.213678,0.181046,0.025388,0.024021
gpaenglish,-0.402801,0.218715,0.181740,-0.083811,-0.186303
gpamath,-0.379170,0.243580,0.116025,0.077460,0.234863
gpaoverall,-0.425672,0.251455,0.229665,-0.043660,-0.034032
motherhighgrade,-0.185914,-0.514389,0.239725,-0.428150,-0.588300
fatherhighgrade,-0.201618,-0.507950,0.179787,-0.346770,0.703983
parentincome,-0.160537,-0.461227,0.279373,0.817850,-0.075447
gender_Male,0.018166,-0.082061,-0.117291,0.037323,0.106830


In [57]:
# The features that drives each components.
for i in range(5):
    print(f"component #{i}:")
    print(components_df.iloc[:, i].map(abs).sort_values(ascending=False).head(3))
    print()

component #0:
gpaoverall    0.425672
gpaenglish    0.402801
gpascience    0.399085
Name: 0, dtype: float64

component #1:
motherhighgrade    0.514389
fatherhighgrade    0.507950
parentincome       0.461227
Name: 1, dtype: float64

component #2:
satverbal       0.609857
satmath         0.560769
parentincome    0.279373
Name: 2, dtype: float64

component #3:
parentincome       0.81785
motherhighgrade    0.42815
fatherhighgrade    0.34677
Name: 3, dtype: float64

component #4:
fatherhighgrade    0.703983
motherhighgrade    0.588300
gpamath            0.234863
Name: 4, dtype: float64



In [58]:
pca.explained_variance_ratio_

array([0.46073387, 0.19036089, 0.09295703, 0.07163009, 0.05328056])

^ The first component accounts for 46% of the variance alone, followed by an additional 19% for the second component. We can use NumPy's `cumsum` method to see how much of feature variance is explained by the five components cumulatively. We can explain 87% of the variance in the 10 features with 5 components.

In [60]:
import numpy as np

np.cumsum(pca.explained_variance_ratio_)

array([0.46073387, 0.65109476, 0.74405179, 0.81568188, 0.86896244])

Let's transform our features in the testing data based on these five principal compnents.

In [61]:
X_train_pca = pca.transform(X_train_enc)
X_train_pca.shape

(634, 5)

In [62]:
X_test_pca = pca.transform(X_test_enc)

rfc = RandomForestClassifier(n_estimators=100, max_depth=2, n_jobs=-1, random_state=0)
rfc.fit(X_train_pca, y_train.values.ravel())

y_pred = rfc.predict(X_test_pca)
accuracy_score(y_test, y_pred)

0.7032967032967034

### Summary

A dimension reduction technique such as PCA can be a good option when the feature selection challenge is that we have highly correlated features and we want to reduce the number of dimensions without substantially reducting the explained variance.